In [1]:
import pandas as pd
import datetime
import requests
import os
import time

def makeLineProtocol(engineLoad, airFlow, rpm, speed, throttle, ay):
    baseString = "telemetry,id=1,make=toyota,model=rav4,year=2022 EngineLoad={},AirFlowRate={},RPM={},Speed={},ThrottlePosition={},Ay={} {}"
    now = datetime.datetime.now()
    timestamp = int(now.timestamp())
    return baseString.format(engineLoad, airFlow, rpm, speed, throttle, ay, timestamp)

In [2]:
INFLUX_TOKEN = os.environ.get('INFLUX_TOKEN')
MEASUREMENT = 'telemetry'
url = "http://localhost:8086/api/v2/write"

params = {
    "org": "localinflux",
    "bucket": "digitaltwin",
    "precision": "s"
}

headers = {
    "Content-Type": "text/plain",
    "Authorization": "Token {}".format(INFLUX_TOKEN)
}

In [3]:
df = pd.read_csv('dataToWrite.csv')
df = df[['EngineLoad','AirFlowRate','RPM','Speed','ThrottlePosition','Ay']]

In [4]:
for i in range(df.shape[0]):  
    engineLoad = df.iloc[i]['EngineLoad']
    airFlow = df.iloc[i]['AirFlowRate']
    rpm = df.iloc[i]['RPM']
    speed = df.iloc[i]['Speed']
    throttle = df.iloc[i]['ThrottlePosition']
    ay = df.iloc[i]['Ay']
    
    payload = makeLineProtocol(engineLoad, airFlow, rpm, speed, throttle, ay)
    response = requests.post(url, params=params, headers=headers, data=payload)
#     print(payload)
    print('Wrote record {}/{}    -    {}'.format(i, df.shape[0], response.status_code))
    time.sleep(1)

Wrote record 0/6628    -    204
Wrote record 1/6628    -    204
Wrote record 2/6628    -    204
Wrote record 3/6628    -    204
Wrote record 4/6628    -    204
Wrote record 5/6628    -    204
Wrote record 6/6628    -    204
Wrote record 7/6628    -    204
Wrote record 8/6628    -    204
Wrote record 9/6628    -    204
Wrote record 10/6628    -    204
Wrote record 11/6628    -    204
Wrote record 12/6628    -    204
Wrote record 13/6628    -    204
Wrote record 14/6628    -    204
Wrote record 15/6628    -    204
Wrote record 16/6628    -    204
Wrote record 17/6628    -    204
Wrote record 18/6628    -    204
Wrote record 19/6628    -    204
Wrote record 20/6628    -    204
Wrote record 21/6628    -    204
Wrote record 22/6628    -    204
Wrote record 23/6628    -    204
Wrote record 24/6628    -    204
Wrote record 25/6628    -    204
Wrote record 26/6628    -    204
Wrote record 27/6628    -    204
Wrote record 28/6628    -    204
Wrote record 29/6628    -    204
Wrote record 30/6628

Wrote record 245/6628    -    204
Wrote record 246/6628    -    204
Wrote record 247/6628    -    204
Wrote record 248/6628    -    204
Wrote record 249/6628    -    204
Wrote record 250/6628    -    204
Wrote record 251/6628    -    204
Wrote record 252/6628    -    204
Wrote record 253/6628    -    204
Wrote record 254/6628    -    204
Wrote record 255/6628    -    204
Wrote record 256/6628    -    204
Wrote record 257/6628    -    204
Wrote record 258/6628    -    204
Wrote record 259/6628    -    204
Wrote record 260/6628    -    204
Wrote record 261/6628    -    204
Wrote record 262/6628    -    204
Wrote record 263/6628    -    204
Wrote record 264/6628    -    204
Wrote record 265/6628    -    204
Wrote record 266/6628    -    204
Wrote record 267/6628    -    204
Wrote record 268/6628    -    204
Wrote record 269/6628    -    204
Wrote record 270/6628    -    204
Wrote record 271/6628    -    204
Wrote record 272/6628    -    204
Wrote record 273/6628    -    204
Wrote record 2

Wrote record 486/6628    -    204
Wrote record 487/6628    -    204
Wrote record 488/6628    -    204
Wrote record 489/6628    -    204
Wrote record 490/6628    -    204
Wrote record 491/6628    -    204
Wrote record 492/6628    -    204
Wrote record 493/6628    -    204
Wrote record 494/6628    -    204
Wrote record 495/6628    -    204
Wrote record 496/6628    -    204
Wrote record 497/6628    -    204
Wrote record 498/6628    -    204
Wrote record 499/6628    -    204
Wrote record 500/6628    -    204
Wrote record 501/6628    -    204
Wrote record 502/6628    -    204
Wrote record 503/6628    -    204
Wrote record 504/6628    -    204
Wrote record 505/6628    -    204
Wrote record 506/6628    -    204
Wrote record 507/6628    -    204
Wrote record 508/6628    -    204
Wrote record 509/6628    -    204
Wrote record 510/6628    -    204
Wrote record 511/6628    -    204
Wrote record 512/6628    -    204
Wrote record 513/6628    -    204
Wrote record 514/6628    -    204
Wrote record 5

Wrote record 727/6628    -    204
Wrote record 728/6628    -    204
Wrote record 729/6628    -    204
Wrote record 730/6628    -    204
Wrote record 731/6628    -    204
Wrote record 732/6628    -    204
Wrote record 733/6628    -    204
Wrote record 734/6628    -    204
Wrote record 735/6628    -    204
Wrote record 736/6628    -    204
Wrote record 737/6628    -    204
Wrote record 738/6628    -    204
Wrote record 739/6628    -    204
Wrote record 740/6628    -    204
Wrote record 741/6628    -    204
Wrote record 742/6628    -    204
Wrote record 743/6628    -    204
Wrote record 744/6628    -    204
Wrote record 745/6628    -    204
Wrote record 746/6628    -    204
Wrote record 747/6628    -    204
Wrote record 748/6628    -    204
Wrote record 749/6628    -    204
Wrote record 750/6628    -    204
Wrote record 751/6628    -    204
Wrote record 752/6628    -    204
Wrote record 753/6628    -    204
Wrote record 754/6628    -    204
Wrote record 755/6628    -    204
Wrote record 7

Wrote record 968/6628    -    204
Wrote record 969/6628    -    204
Wrote record 970/6628    -    204
Wrote record 971/6628    -    204
Wrote record 972/6628    -    204
Wrote record 973/6628    -    204
Wrote record 974/6628    -    204
Wrote record 975/6628    -    204
Wrote record 976/6628    -    204
Wrote record 977/6628    -    204
Wrote record 978/6628    -    204
Wrote record 979/6628    -    204
Wrote record 980/6628    -    204
Wrote record 981/6628    -    204
Wrote record 982/6628    -    204
Wrote record 983/6628    -    204
Wrote record 984/6628    -    204
Wrote record 985/6628    -    204
Wrote record 986/6628    -    204
Wrote record 987/6628    -    204
Wrote record 988/6628    -    204
Wrote record 989/6628    -    204
Wrote record 990/6628    -    204
Wrote record 991/6628    -    204
Wrote record 992/6628    -    204
Wrote record 993/6628    -    204
Wrote record 994/6628    -    204
Wrote record 995/6628    -    204
Wrote record 996/6628    -    204
Wrote record 9

...

Wrote record 5668/6628    -    204
Wrote record 5669/6628    -    204
Wrote record 5670/6628    -    204
Wrote record 5671/6628    -    204
Wrote record 5672/6628    -    204
Wrote record 5673/6628    -    204
Wrote record 5674/6628    -    204
Wrote record 5675/6628    -    204
Wrote record 5676/6628    -    204
Wrote record 5677/6628    -    204
Wrote record 5678/6628    -    204
Wrote record 5679/6628    -    204
Wrote record 5680/6628    -    204
Wrote record 5681/6628    -    204
Wrote record 5682/6628    -    204
Wrote record 5683/6628    -    204
Wrote record 5684/6628    -    204
Wrote record 5685/6628    -    204
Wrote record 5686/6628    -    204
Wrote record 5687/6628    -    204
Wrote record 5688/6628    -    204
Wrote record 5689/6628    -    204
Wrote record 5690/6628    -    204
Wrote record 5691/6628    -    204
Wrote record 5692/6628    -    204
Wrote record 5693/6628    -    204
Wrote record 5694/6628    -    204
Wrote record 5695/6628    -    204
Wrote record 5696/66

Wrote record 5903/6628    -    204
Wrote record 5904/6628    -    204
Wrote record 5905/6628    -    204
Wrote record 5906/6628    -    204
Wrote record 5907/6628    -    204
Wrote record 5908/6628    -    204
Wrote record 5909/6628    -    204
Wrote record 5910/6628    -    204
Wrote record 5911/6628    -    204
Wrote record 5912/6628    -    204
Wrote record 5913/6628    -    204
Wrote record 5914/6628    -    204
Wrote record 5915/6628    -    204
Wrote record 5916/6628    -    204
Wrote record 5917/6628    -    204
Wrote record 5918/6628    -    204
Wrote record 5919/6628    -    204
Wrote record 5920/6628    -    204
Wrote record 5921/6628    -    204
Wrote record 5922/6628    -    204
Wrote record 5923/6628    -    204
Wrote record 5924/6628    -    204
Wrote record 5925/6628    -    204
Wrote record 5926/6628    -    204
Wrote record 5927/6628    -    204
Wrote record 5928/6628    -    204
Wrote record 5929/6628    -    204
Wrote record 5930/6628    -    204
Wrote record 5931/66

Wrote record 6138/6628    -    204
Wrote record 6139/6628    -    204
Wrote record 6140/6628    -    204
Wrote record 6141/6628    -    204
Wrote record 6142/6628    -    204
Wrote record 6143/6628    -    204
Wrote record 6144/6628    -    204
Wrote record 6145/6628    -    204
Wrote record 6146/6628    -    204
Wrote record 6147/6628    -    204
Wrote record 6148/6628    -    204
Wrote record 6149/6628    -    204
Wrote record 6150/6628    -    204
Wrote record 6151/6628    -    204
Wrote record 6152/6628    -    204
Wrote record 6153/6628    -    204
Wrote record 6154/6628    -    204
Wrote record 6155/6628    -    204
Wrote record 6156/6628    -    204
Wrote record 6157/6628    -    204
Wrote record 6158/6628    -    204
Wrote record 6159/6628    -    204
Wrote record 6160/6628    -    204
Wrote record 6161/6628    -    204
Wrote record 6162/6628    -    204
Wrote record 6163/6628    -    204
Wrote record 6164/6628    -    204
Wrote record 6165/6628    -    204
Wrote record 6166/66

Wrote record 6373/6628    -    204
Wrote record 6374/6628    -    204
Wrote record 6375/6628    -    204
Wrote record 6376/6628    -    204
Wrote record 6377/6628    -    204
Wrote record 6378/6628    -    204
Wrote record 6379/6628    -    204
Wrote record 6380/6628    -    204
Wrote record 6381/6628    -    204
Wrote record 6382/6628    -    204
Wrote record 6383/6628    -    204
Wrote record 6384/6628    -    204
Wrote record 6385/6628    -    204
Wrote record 6386/6628    -    204
Wrote record 6387/6628    -    204
Wrote record 6388/6628    -    204
Wrote record 6389/6628    -    204
Wrote record 6390/6628    -    204
Wrote record 6391/6628    -    204
Wrote record 6392/6628    -    204
Wrote record 6393/6628    -    204
Wrote record 6394/6628    -    204
Wrote record 6395/6628    -    204
Wrote record 6396/6628    -    204
Wrote record 6397/6628    -    204
Wrote record 6398/6628    -    204
Wrote record 6399/6628    -    204
Wrote record 6400/6628    -    204
Wrote record 6401/66

Wrote record 6608/6628    -    204
Wrote record 6609/6628    -    204
Wrote record 6610/6628    -    204
Wrote record 6611/6628    -    204
Wrote record 6612/6628    -    204
Wrote record 6613/6628    -    204
Wrote record 6614/6628    -    204
Wrote record 6615/6628    -    204
Wrote record 6616/6628    -    204
Wrote record 6617/6628    -    204
Wrote record 6618/6628    -    204
Wrote record 6619/6628    -    204
Wrote record 6620/6628    -    204
Wrote record 6621/6628    -    204
Wrote record 6622/6628    -    204
Wrote record 6623/6628    -    204
Wrote record 6624/6628    -    204
Wrote record 6625/6628    -    204
Wrote record 6626/6628    -    204
Wrote record 6627/6628    -    204
